In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0041582584381103516
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 13.179936234205758
200000 18.57880254919019
300000 18.285432953415423
400000 17.70818894633184
500000 22.210093531464672
600000 21.105895332215102
700000 18.17077979773447
800000 18.486517942395558
900000 10.879272712854316
1000000 15.086858466506387
1100000 16.920020324227497
1200000 16.66010449533081
1300000 17.29035731740618
1400000 18.811558613195505
1500000 16.511175374702287
1600000 18.221892620803054
1700000 17.285169472113367
1800000 9.056133631843238
1900000 16.657001985313588
2000000 19.51561385635765
2100000 18.76168411941755
2200000 20.035540926436205
2300000 18.687069470781566
2400000 17.912446659201
2500000 19.026064269452533
2600000 18.345291027976902
2700000 7.771695580916207
2800000 18.85091075016821
2900000 17.678320252593657
3000000 17

29400000 14.367925985798784
29500000 14.729064542339133
29600000 15.378392659187373
29700000 16.163857617642336
29800000 16.624802875599077
29900000 9.245167587796509
30000000 14.988339722114771
30100000 16.39676029239367
30200000 16.879316998886626
30300000 16.447597753802626
30400000 8.855890793270353
30500000 15.800389853097998
30600000 16.732899022915323
30700000 15.08854936262905
30800000 15.332515351573045
30900000 14.58659850492114
31000000 10.299754998771792
31100000 9.97335580751893
31200000 14.668448412410228
31300000 16.067213227986688
31400000 16.188583852462916
31500000 16.24584425449381
31600000 16.781134790276386
31700000 9.318786320555578
31800000 15.561389357017994
31900000 16.6794231750711
32000000 15.36674244963515
32100000 16.05482277987956
32200000 7.60023119680825
32300000 17.523591107029695
32400000 15.182850054070254
32500000 16.05479818007306
32600000 14.413660830211043
32700000 13.855420389478596
32800000 6.676904643602629
32900000 13.709604665196196
33000000 

59100000 12.475897268699894
59200000 12.196766039616763
59300000 12.651023557177288
59400000 5.45204808559071
59500000 11.865357417243986
59600000 12.8378008990063
59700000 11.8571010536381
59800000 6.783753720445742
59900000 13.633073809973542
60000000 12.500398633345466
60100000 13.907254534087006
60200000 13.7632840710102
60300000 14.219647225511473
60400000 13.875225046022656
60500000 13.690212023889146
60600000 13.310874678169103
60700000 13.356117412342423
60800000 5.747119684912261
60900000 12.04996036722802
61000000 12.400486750404475
61100000 11.617150640176874
61200000 7.287315509149144
61300000 12.028697866946795
61400000 12.045369423380395
61500000 9.28477934816608
61600000 10.174049742082232
61700000 12.627842853438024
61800000 13.17638713558491
61900000 13.07255441250284
62000000 13.205956451425227
62100000 13.355307665131072
62200000 13.243850644731904
62300000 13.241931862557925
62400000 12.647612720385164
62500000 11.452427797000848
62600000 6.762471445544524
62700000 

88900000 11.395704894742172
89000000 10.208806289859504
89100000 10.298265766373992
89200000 10.002381893767879
89300000 9.119016967363818
89400000 6.61352415068239
89500000 4.300263199700541
89600000 9.475378807433327
89700000 7.339922758595238
89800000 10.12176945566184
89900000 10.63852237426771
90000000 9.84751743502734
90100000 10.123268194514445
90200000 10.832306756720797
90300000 11.263462626693874
90400000 10.829598940354623
90500000 11.390182677238236
90600000 10.587871199300587
90700000 10.028582274522426
90800000 9.808924124376698
90900000 10.439062669236998
91000000 9.571462354584012
91100000 7.566527711965799
91200000 8.091182870178164
91300000 4.479616929584864
91400000 5.904809701640503
91500000 10.260452412632246
91600000 9.944398850117533
91700000 9.894474285278509
91800000 10.411418773596248
91900000 10.01709866309749
92000000 10.320435564615302
92100000 10.821109705626457
92200000 11.766668753997886
92300000 10.516288159974701
92400000 10.629828842950065
92500000 10

118200000 8.16417390935567
118300000 8.291285417745799
118400000 8.286934828665283
118500000 8.281239030027052
118600000 8.235618441409299
118700000 9.130520117047395
118800000 8.960835965623776
118900000 8.576938105259927
119000000 8.178832217440208
119100000 8.196438127018778
119200000 8.222348503534645
119300000 8.16853470875898
119400000 7.883369747387062
119500000 5.154794786239862
119600000 3.899689150545879
119700000 4.404027340940764
119800000 7.140940772650328
119900000 8.02269443078045
120000000 8.246264252634884
120100000 8.042378167697006
120200000 8.478656971995608
120300000 8.20917236626192
120400000 8.340143905494227
120500000 8.96501558740333
120600000 8.69653515110301
120700000 8.205592086332217
120800000 8.234973699592924
120900000 8.240632701468765
121000000 8.373713475045545
121100000 8.275401934792697
121200000 7.518965196771254
121300000 4.598347387502428
121400000 3.6794221239219578
121500000 4.897053631015728
121600000 7.185196019025391
121700000 7.7878312982541

147500000 6.281944060436494
147600000 4.3393515402823795
147700000 5.45007025038409
147800000 6.2514211306239105
147900000 5.91978827877748
148000000 3.2499222186843855
148100000 6.02957534064283
148200000 5.906585764497032
148300000 2.8643330719922018
148400000 6.2436252833001875
148500000 6.3116298888903355
148600000 6.824521179307344
148700000 6.886735620106919
148800000 6.653013719617724
148900000 7.262029020816518
149000000 6.838796457395689
149100000 6.530055977222593
149200000 6.761958300125992
149300000 6.373339981576899
149400000 3.386749926986277
149500000 6.202869948499534
149600000 6.23914936549935
149700000 4.230056578460872
149800000 4.7470923436381
149900000 6.7000748447916445
150000000 6.043542319194874
150100000 3.1951624369703366
150200000 6.857126858564443
150300000 6.637026406395326
150400000 7.149673404760622
150500000 7.134863076073202
150600000 7.41173640154177
150700000 6.5028370677276515
150800000 6.836508210764147
150900000 6.087593389943858
151000000 6.494063

176700000 4.462875007894456
176800000 4.542215073674146
176900000 2.3327140768792334
177000000 4.795578894540071
177100000 4.557360260683938
177200000 4.520660890578952
177300000 4.719127693915764
177400000 4.5260017109358746
177500000 2.765207352429949
177600000 4.534725569855183
177700000 4.232297232007425
177800000 4.336607731926888
177900000 4.139496762353341
178000000 4.107664287051843
178100000 1.9006387112070489
178200000 4.242086700786442
178300000 4.202430877239459
178400000 4.134456500547909
178500000 4.467493846230447
178600000 4.34582820188626
178700000 2.5049464081245825
178800000 4.650236703916341
178900000 4.552238999759412
179000000 4.233075408638344
179100000 4.161996830391615
179200000 3.8363052358669107
179300000 3.3465153328502444
179400000 4.180048954236538
179500000 4.479662600394117
179600000 3.9252292361751397
179700000 3.957618784457604
179800000 3.3535223397359153
179900000 2.4593601094875237
180000000 3.905020851997167
180100000 3.718398169593569
180200000 4.

205600000 1.3554040433299013
205700000 2.5739738548744056
205800000 2.472839894381891
205900000 2.5035185760433243
206000000 2.5010694984810153
206100000 2.371480847919891
206200000 2.4937529548171558
206300000 2.557301233202957
206400000 1.4367516080664098
206500000 2.040169450310154
206600000 2.6097900086764594
206700000 2.4883950122511886
206800000 2.4513016878012515
206900000 2.5272680895222064
207000000 2.4021863891684214
207100000 2.3681071003474856
207200000 1.895514859258029
207300000 1.5465311056503654
207400000 1.5377118262498082
207500000 2.3366449064930412
207600000 2.481405489445617
207700000 2.369803647153828
207800000 2.319981511230691
207900000 2.26470422808524
208000000 2.192586648993075
208100000 2.3543922276396123
208200000 0.9423688264663518
208300000 2.4062088081409896
208400000 2.2316479796533817
208500000 2.264525344410777
208600000 2.195338339802182
208700000 2.3877852993122275
208800000 2.336976747550418
208900000 2.2761491499745317
209000000 1.5887265825480619

233600000 0.1365545244797998
233700000 0.09700805018702151
233800000 0.08217249534809588
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.05176469526723027
234000000 0.013857184019992749
234100000 0.01367004370991389
234200000 0.013090693922794529
234300000 0.012748701403796673
234400000 0.0123677934580942
234500000 0.012006888091537687
234600000 0.011405184285402298
234700000 0.011166343622151348
234800000 0.010899132182535198
234900000 0.010517747850944597
235000000 0.010172575595988166
235100000 0.009846982263574998
235200000 0.011375845649152996
235300000 0.015397452810403374
235400000 0.01423125390003787
235500000 0.01410461070684592
235600000 0.013610029125975238
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.04656433926453193
235800000 0.0311272214286228
235900000 0.027664018196668888
236000000 0.03497397070258524
236100000 0.037080203564733266
236200000 0.011378121594054831
236300000 0.010549253248386913
2364